In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')

tf.config.experimental.set_virtual_device_configuration(
    gpus[0], 
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*5)]
)

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
import numpy as np
import pandas as pd

from utils.Block import Model, Block
from utils.train import train

import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import make_scorer
from sklearn import ensemble, linear_model
from xgboost.sklearn import XGBClassifier

from catboost import Pool, CatBoostClassifier

X_submit = pd.concat([ 
    pd.read_csv('./data/420_X_submit.csv'), 
    pd.read_csv('./data/510_X_submit_preds.csv', header=None),
    pd.read_csv('./data/520_X_submit_preds.csv', header=None),
], axis=1)

X_train = pd.concat([
    pd.read_csv('./data/420_X_train.csv'), 
    pd.read_csv('./data/510_X_train_preds.csv', header=None),
    pd.read_csv('./data/520_X_train_preds.csv', header=None)
], axis=1)

X_test = pd.concat([
    pd.read_csv('./data/420_X_test.csv'),
    pd.read_csv('./data/510_X_test_preds.csv', header=None), 
    pd.read_csv('./data/520_X_test_preds.csv', header=None), 
], axis=1)

X_submit = X_submit.set_index('id')
X_train  = X_train.set_index('id')
X_test   = X_test.set_index('id')


y_submit = pd.read_csv('./data/004_test.csv', index_col='id')
y_train = pd.read_csv('./data/410_y_train.csv', index_col='id')
y_test = pd.read_csv('./data/410_y_test.csv', index_col='id')


y_train = pd.get_dummies(y_train['label'])
y_test = pd.get_dummies(y_test['label'])

X_submit = X_submit.values
X_train = X_train.values
X_test = X_test.values
y_train = np.squeeze(y_train.values)
y_test = np.squeeze(y_test.values)

X_train.shape

(80000, 427)

In [3]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [4]:
X_train = tf.convert_to_tensor(X_train, dtype='float64')
y_train = tf.convert_to_tensor(y_train, dtype='float64')
X_test  = tf.convert_to_tensor(X_test, dtype='float64')
y_test  = tf.convert_to_tensor(y_test, dtype='float64')
X_submit = tf.convert_to_tensor(X_submit, dtype='float64')

In [5]:
class Model(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.block1 = Block(128)
        self.block2  = Block(128)
        self.out = tf.keras.layers.Dense(13, dtype='float64')
        
    def call(self, X):
        X = self.block1(X)
        X = self.block2(X)
        X = self.out(X)
        return X

In [6]:
y_train

<tf.Tensor: id=2, shape=(80000, 13), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])>

In [7]:
model = Model()

In [8]:
model = train(model, X_train, y_train, X_test, y_test, batch_size=5000)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch: 0 	 train

In [28]:
model_name = '610_stack_nn'
model.save_weights('M_336/checkpoints/'+model_name+'/model')

# Predict Submission

In [9]:
X_test_pred   = np.squeeze(np.array(model(X_test)))
X_submit_pred = np.squeeze(np.array(model(X_submit)))


X_test_pred.shape

(20000, 13)

In [10]:
df_submit = pd.DataFrame.from_records(tf.nn.softmax(X_submit_pred).numpy())

cols = ['class'+str(i) for i in range(13)]

df_submit.columns = cols
df_submit.head()

,class0,class1,class2,class3,class4,class5,class6,class7,class8,class9,class10,class11,class12
0,0.002025,0.016797,0.215486,0.034620,0.056751,8.248962e-05,0.003916,0.008444,0.012151,0.008461,0.615139,0.003955,0.022171
1,0.001065,0.013098,0.604899,0.187896,0.123062,2.888404e-06,0.003948,0.008679,0.003639,0.005173,0.046217,0.000813,0.001508
2,0.000945,0.073293,0.207101,0.579528,0.000567,3.105479e-07,0.019966,0.002881,0.034359,0.007722,0.057693,0.000369,0.015576
3,0.001720,0.089113,0.242712,0.220007,0.259783,1.861609e-06,0.014547,0.024224,0.018379,0.016635,0.096587,0.005773,0.010520
4,0.001444,0.000831,0.032817,0.004558,0.028877,5.164665e-07,0.001377,0.000452,0.001240,0.000662,0.916314,0.000058,0.011369


In [17]:
df_submit = pd.concat([
    y_submit.reset_index(),
    df_submit
], axis=1)

df_submit = df_submit.set_index('id')
df_submit

,class0,class1,class2,class3,class4,class5,class6,class7,class8,class9,class10,class11,class12
id,,,,,,,,,,,,,
151807,0.014811,0.012477,0.211700,0.101431,0.114950,7.378615e-06,0.003703,0.024584,0.004041,0.005654,0.351503,0.003975,0.151164
118131,0.000258,0.015453,0.405367,0.340845,0.083052,4.245976e-07,0.004999,0.068152,0.011348,0.013953,0.049724,0.000234,0.006617
110921,0.001428,0.018323,0.221510,0.687187,0.007387,4.925802e-07,0.004762,0.003744,0.012540,0.001526,0.019810,0.000331,0.021452
105149,0.000448,0.041217,0.289852,0.154545,0.340828,5.808531e-07,0.001026,0.051054,0.002478,0.002529,0.099349,0.001727,0.014948
143868,0.004046,0.004195,0.113199,0.063066,0.215284,4.050218e-06,0.001415,0.000359,0.002075,0.000393,0.583034,0.001678,0.011251
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146316,0.004989,0.027021,0.180711,0.304243,0.302709,3.391095e-07,0.007697,0.030366,0.020216,0.010280,0.026087,0.000294,0.085388
121816,0.003338,0.030038,0.198639,0.098427,0.366782,1.087257e-05,0.004343,0.000250,0.002856,0.002260,0.285694,0.000349,0.007013
106217,0.001167,0.016636,0.019767,0.130547,0.731146,9.241584e-07,0.002916,0.000458,0.006342,0.008682,0.075869,0.000110,0.006358


In [18]:
df_submit.to_csv('005_submit.csv')